# STA 220 Assignment 1

Due __January 26, 2024__ by __11:59pm__. Submit your work by uploading it to Gradescope through Canvas.

Instructions:

1. Provide your solutions in new cells following each exercise description. Create as many new cells as necessary. Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
2. The use of assistive tools is permitted, but must be indicated. You will be graded on you proficiency in coding. Produce high quality code by adhering to proper programming principles. 
3. Export the .jpynb as .pdf and submit it on Gradescope in time. To facilitate grading, indicate the area of the solution on the submission. Submissions without indication will be marked down. No late submissions accepted. 
4. If test cases are given, your solution must be in the same format. 
5. The total number of points is 10. 

__Exercise 1__

Answer the following questions by querying [Lahman Baseball Database](http://seanlahman.com/). The 2019 version together with the description (`readme2019.txt`) are on Piazza. Answer the following questions. 

The purpose of this assignment is to practice accessing and analyzing data in a database. For full credit for (a) and (b), query the correct table with `pandas.read_sql` and a single SQL query. Unless otherwise specified, return in the same format of the test case. 

__(a, i)__ Which pitcher has the second most [home runs allowed](https://en.wikipedia.org/wiki/Home_runs_allowed) in the American League? __(ii)__ Which pitcher has the worst home runs allowed per game ratio? 

In [4]:
import pandas as pd
import sqlite3 as sql

In [5]:
db = sql.connect("../Data/lahmansbaseballdb.sqlite")

In [15]:
teams = pd.read_sql("""SELECT * FROM Teams""", db)
teams['lgID'].unique()

array(['NA', 'NL', 'AA', 'UA', 'PL', 'AL', 'FL'], dtype=object)

In [58]:
# most home runs allowed in the American League: 
result = pd.read_sql("""SELECT nameFirst, nameLast, sum(hr) as HR FROM Pitching as pi
                     LEFT JOIN People as p
                     on pi.playerID = p.playerID
                     WHERE lgID = 'AL'
                     GROUP BY pi.playerID
                     ORDER BY sum(hr) DESC
                     LIMIT 1
                     OFFSET 1""", db).squeeze()
print(result['nameFirst'] + " " + result['nameLast'] + " (" + str(result["HR"]) + ")")

Tim Wakefield (401)


In [74]:
result = pd.read_sql("""SELECT nameFirst, nameLast, SUM(HR) as HRS, TOTAL(HR) as HR, TOTAL(G) as G, (TOTAL(HR) / TOTAL(G)) AS Ratio FROM Pitching as pi
                     LEFT JOIN People as p
                     on pi.playerID = p.playerID
                     WHERE lgID = 'AL'
                     GROUP BY pi.playerID
                     ORDER BY RATIO DESC
                     LIMIT 1""", db ).squeeze()
print(result['nameFirst'] + " " + result['nameLast'] + " (" + str(result["Ratio"]) + ")")

Ryan Snare (3.0)


__(b, i)__ Amongst all players in the American League that have passed, report their average lifespan in full years. __(ii)__ Return the six schools with most hall-of-fame alumni. __(iii)__ What fraction of managers have not been professional players?  

In [99]:
# average lifespan for passed players in the National League
result = pd.read_sql("""SELECT AVG(deathYear - birthYear) as avglifespan FROM People
                     WHERE deathYear IS NOT NULL AND """, db).squeeze()
round(result)

68

In [138]:
result = pd.read_sql("""SELECT Sc.name_full, COUNT(HoF.playerID) as Count FROM HallofFame as HoF
                     INNER JOIN CollegePlaying as CP
                     ON HoF.playerID = CP.playerID
                     INNER JOIN Schools as Sc
                     ON CP.schoolID = Sc.schoolID
                     WHERE inducted = 'Y'
                     GROUP BY CP.schoolID
                     ORDER BY COUNT(HoF.playerID) DESC
                     LIMIT 6""", db).squeeze()
result

,name_full,Count
0,St. Bonaventure University,7
1,University of Michigan,6
2,University of Minnesota,6
3,University of Southern California,6
4,San Diego State University,5
5,University of Alabama,4


__(c)__ Create a world map with a color gradient corresponding to the log-number of players per country in the data set. Use `pandas.read_html` to retrieve the ISO codes from [wikipedia](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes), and merge those records as good as you can. Match no more than ten countries to their ISOs manually. 

__Exercise 2__

We will use the [lichess](https://lichess.org/api) API to retrieve some information about the current state of chess in the world. In order to answer below questions, make precise and economical requests. You may use:
```
import requests
import json
import pandas

from datetime import datetime
```

__(a)__ What is the real name of the leader of the blitz leaderboard? 

__(b, i)__ Get the username of the last player that played a rapid game against user `athena-pallada`. __(ii)__ In all games against this user, how many times did `athena-pallada` win? _(Provide code that answers the question in case more than just a single game is returned)_

__(c)__ Consider the top ten players in the bullet leaderboard. __(i)__ Which player has the most bullet games overall? __(ii)__ Which player has played the most bullet games relative to account age in days? __(iii)__ Which player has the worst win-to-loss ratio over all formats?

__(d)__ Get all games from user `manwithavan`. Group them by opening and print the ten most popular. 

In [295]:
openings.value_counts().head(2)

Van't Kruijs Opening                     7
Nimzo-Larsen Attack: Modern Variation    7
dtype: int64